# <span style="color:#0b486b">  FIT5215: Deep Learning (T3, 2020)</span>
***
*Chief Examiner:* Prof **Dinh Phung** | dinh.phung@monash.edu <br/>
*Lecturer:* Dr **Peibo Duan** | peibo.duan@monash.edu <br/>
*Tutor:* Mr **Qing Ye** | qing.ye@monash.edu <br/>

<br/>
Department of Data Science and AI, Faculty of Information Technology, Monash University, Australia <br/>

***

## <span style="color:#0b486b">Assignment 02: Neural Embedding and Sequence Modelling</span>
### Due: <span style="color:red">11:59pm 29 August 2020</span>  (Sunday)

#### <span style="color:red">Important note:</span> This is an **individual** assignment. It contributes **20%** to you final mark. Read the assignment instruction carefully.

## <span style="color:#0b486b">Instructions</span>

This notebook has been prepared for your to complete Assignment 2. The theme of this assignment is about practical machine learning knowledge and skills in deep neural networks, word embedding and text analytics. Some sections have been partially completed to help you get
started. **The total marks for this notebook is 110**.

* Before you start, read the entire notebook carefully once to understand what you need to do. <br><br>
* For each cell marked with **#YOU ARE REQUIRED TO INSERT YOUR CODES IN THIS CELL**, there will be places where you **must** supply your own codes when instructed. <br>

This assignment contains **four** parts:

* Part 1: Questions on downloading and preprocessing data **[10 points]**
* Part 2: Questions on using Word2Vect to transform texts to vectors **[20 points]**
* Part 3: Coding assessment on Text CNN for sequence modeling and neural embedding **[15 points]**
* Part 4: Coding assessment on RNNs for sequence modeling and neural embedding **[65 points]**


**Hint**: This assignment was essentially designed based on the lectures and tutorials sessions covered from Week 5 to 6. You are strongly recommended to go through these contents thoroughly which might help you to complete this assignment.

## <span style="color:#0b486b">What to submit</span>

This assignment is to be completed individually and submitted to Moodle unit site. **By the due date, you are required to submit one  <span style="color:red; font-weight:bold">single zip file, named xxx_assignment02_solution.zip</span> where `xxx` is your student ID, to the corresponding Assignment (Dropbox) in Moodle**. 

***For example, if your student ID is <span style="color:red; font-weight:bold">12356</span>, then gather all of your assignment solution to folder, create a zip file named <span style="color:red; font-weight:bold">123456_assignment02_solution.zip</span> and submit this file.***

Within this zip folder, you **must** submit the following files:
1.	**Assignment02_solution.ipynp**:  this is your Python notebook solution source file.
1.	**Assignment02_output.html**: this is the output of your Python notebook solution *exported* in html format.
1.	Any **extra files or folder** needed to complete your assignment (e.g., images used in your answers).

## <span style="color:#0b486b">Part 1: Download and preprocess the data</span>

<div style="text-align: right"><span style="color:red; font-weight:bold">[Total marks for this part: 10 points]<span></div>

The dataset we use for this assignment is a small [Question Classification](http://cogcomp.org/Data/QA/QC/) dataset which was used in the research paper on the topic of ***Question Classification*** (QC) published by Roth and Xin in 2002 (which has received close to 1000 citations till today). Note that a pdf copy of this paper has been downloaded into this assignment folder (**`Roth_Xin_coling02.pdf`**).

The train set consists of $1,000$ questions belonging to 6 coarse question categories, including:
- abbreviation (ABBR), 
- entity (ENTY), 
- description (DESC), 
- human (HUM), 
- location (LOC) and 
- numeric (NUM).


Preprocessing data is an inital and important step in any machine learning or deep learning projects. The following *DataManager* class helps you to download data and preprocess data for the later steps of a deep learning project. 

In [1]:
import os
import zipfile
import collections
import numpy as np
import tensorflow as tf
from six.moves import range
from six.moves.urllib.request import urlretrieve
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tensorflow import keras
plt.style.use('ggplot')

class DataManager:
    def __init__(self, verbose=True, maxlen= 50, random_state=6789):
        self.verbose = verbose
        self.max_sentence_len = 0
        self.str_questions = list()
        self.str_labels = list()
        self.numeral_labels = list()
        self.maxlen = maxlen
        self.numeral_data = list()
        self.random_state = random_state
        self.random = np.random.RandomState(random_state)
        
    @staticmethod
    def maybe_download(dir_name, file_name, url, verbose= True):
        if not os.path.exists(dir_name):
            os.mkdir(dir_name)
        if not os.path.exists(os.path.join(dir_name, file_name)):
            urlretrieve(url + file_name, os.path.join(dir_name, file_name))
        if verbose:
            print("Downloaded successfully {}".format(file_name))
    
    def read_data(self, dir_name, file_names):
        for file_name in file_names:
            file_path= os.path.join(dir_name, file_name)
            self.str_questions= list(); self.str_labels= list()
            with open(file_path, "r", encoding="latin-1") as f:
                for row in f:
                    row_str= row.split(":")
                    label, question= row_str[0], row_str[1]
                    question= question.lower()
                    self.str_labels.append(label)
                    self.str_questions.append(question[0:-1])
                    if self.max_sentence_len < len(self.str_questions[-1]):
                        self.max_sentence_len= len(self.str_questions[-1])
         
        # turns labels into numbers
        le= preprocessing.LabelEncoder()
        le.fit(self.str_labels)
        self.numeral_labels = np.array(le.transform(self.str_labels))
        self.str_classes= le.classes_
        self.num_classes= len(self.str_classes)
        if self.verbose:
            print("\nSample questions... \n")
            print(self.str_questions[0:5])
            print("Labels {}\n\n".format(self.str_classes))
    
    def manipulate_data(self):
        tokenizer = tf.keras.preprocessing.text.Tokenizer()
        tokenizer.fit_on_texts(self.str_questions)
        self.numeral_data = tokenizer.texts_to_sequences(self.str_questions)
        self.numeral_data = tf.keras.preprocessing.sequence.pad_sequences(self.numeral_data, padding='post', truncating= 'post', maxlen= self.maxlen)
        self.word2idx = tokenizer.word_index
        self.word2idx = {k:v for k,v in self.word2idx.items()}
        self.idx2word = {v:k for k,v in self.word2idx.items()}
        self.vocab_size = len(self.word2idx)
    
    def train_valid_split(self, train_ratio=0.9):
        idxs = np.random.permutation(np.arange(len(self.str_questions)))
        train_size = int(train_ratio*len(idxs)) +1
        self.train_str_questions, self.valid_str_questions = self.str_questions[0:train_size], self.str_questions[train_size:]
        self.train_numeral_data, self.valid_numeral_data = self.numeral_data[0:train_size], self.numeral_data[train_size:]
        self.train_numeral_labels, self.valid_numeral_labels = self.numeral_labels[0:train_size], self.numeral_labels[train_size:]
        self.tf_train_set = tf.data.Dataset.from_tensor_slices((self.train_numeral_data, self.train_numeral_labels))
        self.tf_valid_set = tf.data.Dataset.from_tensor_slices((self.valid_numeral_data, self.valid_numeral_labels))

In [2]:
print('Loading data...')
DataManager.maybe_download("Data", "train_1000.label", "http://cogcomp.org/Data/QA/QC/")
DataManager.maybe_download("Data", "TREC_10.label", "http://cogcomp.org/Data/QA/QC/")

dm = DataManager(maxlen=100)
dm.read_data("Data/", ["train_1000.label", "TREC_10.label"])   # read data

Loading data...
Downloaded successfully train_1000.label
Downloaded successfully TREC_10.label

Sample questions... 

['dist how far is it from denver to aspen ?', 'city what county is modesto , california in ?', 'desc who was galileo ?', 'def what is an atom ?', 'date when did hawaii become a state ?']
Labels ['ABBR' 'DESC' 'ENTY' 'HUM' 'LOC' 'NUM']




In [3]:
dm.manipulate_data()
dm.train_valid_split(train_ratio=0.8)

You now have a data manager, named *dm* containing the training and validiation sets in both text and numeric forms. Your task is to play around and read this code to figure out the meanings of some important attributes that will be used in the next parts.

#### <span style="color:red">**Question 1.1**</span> 
**What is the purpose of `self.train_str_questions` and `self.train_numeral_labels`? Write your code to print out the first five questions with labels in the training set.**

<div style="text-align: right"><span style="color:red">[2 points]</span></div> 

#Your answer here

self.train_str_quesiton is a list which stores the questions of the training data in the string forms.
self.train_numeral_labels is a list which store the labels of the training data in digit forms.


In [4]:
#Your code here
for i in range(5):
    print(dm.train_str_questions[i])
    print(dm.str_classes[dm.train_numeral_labels[i]])
    print("*****************************")

dist how far is it from denver to aspen ?
NUM
*****************************
city what county is modesto , california in ?
LOC
*****************************
desc who was galileo ?
HUM
*****************************
def what is an atom ?
DESC
*****************************
date when did hawaii become a state ?
NUM
*****************************


#### <span style="color:red">**Question 1.2**</span> 
**What is the purpose of `self.train_numeral_data`? Write your code to print out the first five questions in the numeric format with labels in the training set.**

<div style="text-align: right"><span style="color:red">[2 points]</span></div> 

#Your answer here

If we want to input the data to the model to train, we can input string directly, so we input the index of each word in the vocabulary table. 
self.train_numeral_data is the list which stores the index of each word in the training questions.

In [5]:
#Your code here
for i in range(5):
    print(dm.train_numeral_data[i])
    print(dm.str_classes[dm.train_numeral_labels[i]])
    print("*****************************")

[ 27  14  73   3 115  39 305  20 306   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0]
NUM
*****************************
[ 18   1 153   3 307 308   6   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0]
LOC
*****************************
[ 42  12   8 309   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0 

#### <span style="color:red">**Question 1.3**</span> 
**What is the purpose of two dictionaries: `self.word2idx` and `self.idx2word`? Write your code to print out the first five key-value pairs of those dictionaries.**

<div style="text-align: right"><span style="color:red">[2 points]</span></div> 

#Your answer here

self.word2idx stores the index of each word, if we want to know the index of word x, we could find it using the code 'self.word2idx[x]'.
self.word2idx can convert the index to word, if we want to know the word with index i, we could find it using the code 'self.idx2word[i]'.




In [6]:
#Your code here
for i in range(5):
    print('index:', i+1 ,'word:', dm.idx2word[i+1])


index: 1 word: what
index: 2 word: the
index: 3 word: is
index: 4 word: def
index: 5 word: of


#### <span style="color:red">**Question 1.4**</span> 
**What is the purpose of `self.tf_train_set`? Write your code to print out the first five items of `self.tf_train_set`.**

<div style="text-align: right"><span style="color:red">[2 points]</span></div> 

#Your answer here

We can not use tf2 to train data in the array form, so we must transform them to TensorSliceDataset. In the TensorSliceDataset, there are tensors which is accepted by tf2.

In [7]:
#Your code here
i = 0
for tf in dm.tf_train_set:
    i += 1
    print(tf)
    if i == 5:
        break


(<tf.Tensor: shape=(100,), dtype=int32, numpy=
array([ 27,  14,  73,   3, 115,  39, 305,  20, 306,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0])>, <tf.Tensor: shape=(), dtype=int32, numpy=5>)
(<tf.Tensor: shape=(100,), dtype=int32, numpy=
array([ 18,   1, 153,   3, 307, 308,   6,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   

#### <span style="color:red">**Question 1.5**</span> 
**What is the purpose of `self.tf_valid_set`? Write your code to print out the first five items of `self.tf_valid_set`.**

<div style="text-align: right"><span style="color:red">[2 points]</span></div> 

#Your answer here

We can not use tf2 to valid data in the array form, so we must transform them to TensorSliceDataset. In the TensorSliceDataset, there are tensors which is accepted by tf2.


In [8]:
#Your code here

i = 0
for tf in dm.tf_valid_set:
    i += 1
    print(tf)
    if i == 5:
        break


(<tf.Tensor: shape=(100,), dtype=int32, numpy=
array([ 32,   1,   3,   2, 921, 922, 923, 924,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0])>, <tf.Tensor: shape=(), dtype=int32, numpy=2>)
(<tf.Tensor: shape=(100,), dtype=int32, numpy=
array([ 11,  17,   3, 925,  15,  69,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   

## <span style="color:#0b486b">Part 2: Using Word2Vect to transform texts to vectors </span>

<div style="text-align: right"><span style="color:red; font-weight:bold">[Total marks for this part: 20 points]<span></div>

In this part, you will be assessed on how to use a pretrained Word2Vect model for realizing a machine learning task. Basically, you will use this pretrained Word2Vect to transform the questions in the above dataset stored in the *data manager object dm* to numeric form for training a Support Vector Machine in sckit-learn.  

In [9]:
import gensim.downloader as api
from gensim.models import Word2Vec
from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score

#### <span style="color:red">**Question 2.1**</span> 
**Write code to download the pretrained model *glove-wiki-gigaword-100*. Note that this model transforms a word in its dictionary to a $100$ dimensional vector.**

<div style="text-align: right"><span style="color:red">[2 points]</span></div> 

In [10]:
word2vect = api.load("glove-wiki-gigaword-100")#Insert your code here

#### <span style="color:red">**Question 2.2**</span> 

**Write code for the function *get_word_vector(word, model)* used to transform a word to a vector using the pretrained Word2Vect model *model*. Note that for a word not in the vocabulary of our *word2vect*, you need to return a vector $0$ with 100 dimensions.**

<div style="text-align: right"><span style="color:red">[3 points]</span></div> 

In [11]:
def get_word_vector(word, model):
    try:
        vector = model[word]#Insert your code here
    except: #word not in the vocabulary
        vector = np.zeros((100), dtype=np.float32)#Insert your code here
    return vector

#### <span style="color:red">**Question 2.3**</span> 

**Write the code for the function *get_sentence_vector(sentence, important_score=None, model= None)*. Note that this function will transform a sentence to a 100-dimensional vector using the pretrained model *model*. In addition, the list *important_score* which has the same length as the *sentence* specifies the important scores of the words in the sentence. In your code, you first need to apply *softmax* function over *important_score* to obtain the important weight *important_weight* which forms a probability over the words of the sentence. Furthermore, the final vector of the sentence will be weighted sum of the individual vectors for words and the weights in *important_weight*.**
- $final\_vector= important\_weight[1]\times v[1] + important\_weight[2]\times v[2] + ...+ important\_weight[L]\times v[L]$ where $L$ is the length of the sentence and $v[i]$ is the vector of the word $i-th$ in this sentence.

**Note that if *important_score=None* is set by default, your function should return the average of all representation vectors corresponding to set *important_score=[1,1,...,1]*.**

<div style="text-align: right"><span style="color:red">[5 points]</span></div> 

In [12]:
def get_sentence_vector(sentence, important_score=None, model= None):
    #Insert your code here
    final_vector = np.zeros((100), dtype=np.float32)
    words = sentence.split(' ')
    for i in range(len(words)):
        final_vector += important_score[i] * get_word_vector(words[i], model)
    return final_vector


#### <span style="color:red">**Question 2.4**</span> 

**Write code to transform the training questions in *dm.train_str_questions* to feature vectors. Note that after running the following cell, you must have $X\_train$ which is an numpy array of the feature vectors and $y\_train$ which is an array of numeric labels (*Hint: dm.train_numeral_labels*). You can add more lines to the following cell if necessary. In addition, you should decide the *important_score* by yourself. For example, you might reckon that the 1st score is 1, the 2nd score is decayed by 0.9, the 3rd is decayed by 0.9, and so on.**

<div style="text-align: right"><span style="color:red">[2.5 points]</span></div> 

In [13]:
print("Transform training set to feature vectors...")
important_score = np.array([0.9**i for i in range(100)])
X_train= [get_sentence_vector(line, important_score, word2vect) for line in dm.train_str_questions]#Insert your code here
y_train= dm.train_numeral_labels#Insert your code here

Transform training set to feature vectors...


#### <span style="color:red">**Question 2.5**</span> 

**Write code to transform the training questions in *dm.valid_str_questions* to feature vectors. Note that after running the following cell, you must have $X\_valid$ which is an numpy array of the feature vectors and $y\_valid$ which is an array of numeric labels (*Hint: dm.valid_numeral_labels*). You can add more lines to the following cell if necessary. In addition, you should decide the *important_score* by yourself. For example, you might reckon that the 1st score is 1, the 2nd score is decayed by 0.9, the 3rd is decayed by 0.9, and so on.**

<div style="text-align: right"><span style="color:red">[2.5 points]</span></div> 

In [14]:
print("Transform valid set to feature vectors...")
important_score = np.array([0.9**i for i in range(100)])
X_valid= [get_sentence_vector(line, important_score, word2vect) for line in dm.valid_str_questions]#Insert your code here
y_valid= dm.valid_numeral_labels#Insert your code here

Transform valid set to feature vectors...


#### <span style="color:red">**Question 2.6**</span> 

**It is now to use *MinMaxScaler(feature_range=(-1,1))* in sckit-learn to scale both training and valid sets to the range $(-1,1)$.**

<div style="text-align: right"><span style="color:red">[2 points]</span></div> 

In [15]:
scaler = MinMaxScaler(feature_range=(-1,1))
scaler.fit(X_train)
X_train = scaler.fit_transform(X_train)#Insert your code here
scaler.fit(X_valid)
X_valid = scaler.fit_transform(X_valid)#Insert your code here

#### <span style="color:red">**Question 2.7**</span> 

**Declare a support vector machine (the class *SVC*  in sckit-learn) with RBF kernel, $C=1$, $gamma= 2^{-3}$ and fit on the training set.**

<div style="text-align: right"><span style="color:red">[3 points]</span></div> 

In [16]:
svm = SVC(C=1.0, gamma=2**-3, kernel='rbf')
#Insert your code here
svm.fit(X_train, y_train)
#Insert your code for fitting svm on X_train

SVC(gamma=0.125)

#### <span style="color:red">**Question 2.8**</span> 

**Finally, we use the trained *svm* to evaluate on the valid set $X\_valid$.**

<div style="text-align: right"><span style="color:red">[2 points]</span></div> 

In [17]:
y_valid_pred = svm.predict(X_valid)#Insert your code
acc = sum(y_valid_pred == y_valid)/len(y_valid_pred)#Insert your code
print(acc)

0.9494949494949495


## <span style="color:#0b486b">Part 3: Text CNN for sequence modeling and neural embedding </span>

<div style="text-align: right"><span style="color:red; font-weight:bold">[Total marks for this part: 15 points]<span></div>

#### <span style="color:red">**Question 3.1**</span> 

**In what follows, you are required to complete the code for Text CNN for sentence classification. The paper of Text CNN can be found at this [link](https://www.aclweb.org/anthology/D14-1181.pdf). Here is the description of the Text CNN you need to construct.**
- There are three attributes (properties or instance variables): *embed_size, state_size, data_manager*.
  - `embed_size`: the dimension of the vector space for which the words are embedded to using the embedding matrix.
  - `state_size`: the number of filters used in *Conv1D* (reference [here](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv1D)).
  - `data_manager`: the data manager to store information of the dataset.
- The detail of the computational process is as follows:
  - Given input $x$, we embed $x$ using the embedding matrix to obtain an $3D$ tensor $[batch\_size \times vocab\_size \times embed\_size]$ as $h$.
  - We feed $h$ to three Convd 1D layers, each of which has $state\_size$ filters, padding=same, activation= relu, and $kernel\_size= 3, 5, 7$ respectively to obtain $h1, h2, h3$. Note that each $h1, h2, h3$ is a 3D tensor with the shape $[batch\_size \times output\_size \times state\_size]$.
  - We then apply *GlobalMaxPool1D()* (reference [here](https://www.tensorflow.org/api_docs/python/tf/keras/layers/GlobalMaxPool1D)) over $h1, h2, h3$ to obtain 2D tensors stored in $h1, h2, h3$ again.
  - We then concatenate three 2D tensors $h1, h2, h3$ to obtain $h$. Note that you need to specify the axis to concatenate.
  - We finally build up one dense layer on the top of $h$ for classification.
  
  <div style="text-align: right"><span style="color:red">[10 points]</span></div>
  

In [18]:
import tensorflow as tf

In [19]:
class TextCNN:
    def __init__(self, embed_size= 128, state_size=16, data_manager=None):
        self.data_manager = data_manager
        self.embed_size = embed_size
        self.state_size = state_size
    
    def build(self):
        x = tf.keras.layers.Input(shape=[100])
        h = tf.keras.layers.Embedding(self.data_manager.vocab_size +1, self.embed_size)(x)
        h1 = tf.keras.layers.Conv1D(self.state_size, 3, padding='same', activation='relu')(h) #Insert your code here
        h2 = tf.keras.layers.Conv1D(self.state_size, 5, padding='same', activation='relu')(h)#Insert your code here
        h3 = tf.keras.layers.Conv1D(self.state_size, 7, padding='same', activation='relu')(h)#Insert your code here
        h1 = tf.keras.layers.GlobalMaxPool1D()(h1)#Insert your code here
        h2 = tf.keras.layers.GlobalMaxPool1D()(h2)#Insert your code here
        h3 = tf.keras.layers.GlobalMaxPool1D()(h3)#Insert your code here
        h = tf.keras.layers.Concatenate(axis=1)([h1, h2, h3])#Insert your code here
        h = tf.keras.layers.Dense(self.data_manager.num_classes, activation='softmax')(h)
        self.model = tf.keras.Model(inputs=x, outputs=h) 
    
    def compile_model(self, *args, **kwargs):
        self.model.compile(*args, **kwargs)
    
    def fit(self, *args, **kwargs):
        return self.model.fit(*args, **kwargs)
    
    def evaluate(self, *args, **kwargs):
        self.model.evaluate(*args, **kwargs)


#### <span style="color:red">**Question 3.2**</span> 
**Here is the code to test TextCNN above. You can observe that TextCNN outperforms the traditional approach SVM + Word2Vect for this task. The reason is that TextCNN enables us to automatically learn the feature that fits to the task. This makes deep learning different from hand-crafted feature approaches. Complete the code to test the model. Note that when compiling the model, you can use the Adam optimizer.**

<div style="text-align: right"><span style="color:red">[5 points]</span></div>

In [20]:
text_cnn = TextCNN(data_manager=dm)
text_cnn.build()
text_cnn.compile_model(optimizer = keras.optimizers.Adam(lr=10e-4),
                                loss = keras.losses.SparseCategoricalCrossentropy(),
                                metrics=['accuracy'])#Insert code here to compile the model
text_cnn.fit(dm.train_numeral_data, dm.train_numeral_labels, 
    validation_data = (dm.valid_numeral_data, dm.valid_numeral_labels),
    epochs=20)#Insert code here to train the model on 20 epochs

Epoch 1/20


F:\Anaconda3\envs\tf2\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


13/13 [==============================] - 4s 32ms/step - loss: 1.7030 - accuracy: 0.3491 - val_loss: 1.6606 - val_accuracy: 0.3333
Epoch 2/20
13/13 [==============================] - 0s 7ms/step - loss: 1.4895 - accuracy: 0.5137 - val_loss: 1.4930 - val_accuracy: 0.4242
Epoch 3/20
13/13 [==============================] - 0s 7ms/step - loss: 1.2641 - accuracy: 0.6509 - val_loss: 1.2876 - val_accuracy: 0.4949
Epoch 4/20
13/13 [==============================] - 0s 8ms/step - loss: 0.9973 - accuracy: 0.8055 - val_loss: 1.0133 - val_accuracy: 0.7374
Epoch 5/20
13/13 [==============================] - 0s 8ms/step - loss: 0.7241 - accuracy: 0.9576 - val_loss: 0.7575 - val_accuracy: 0.8788
Epoch 6/20
13/13 [==============================] - 0s 7ms/step - loss: 0.4991 - accuracy: 0.9601 - val_loss: 0.5910 - val_accuracy: 0.8990
Epoch 7/20
13/13 [==============================] - 0s 7ms/step - loss: 0.3470 - accuracy: 0.9751 - val_loss: 0.4842 - val_accuracy: 0.8990
Epoch 8/20
13/13 [============

## <span style="color:#0b486b">Part 4: RNNs for sequence modeling and neural embedding </span>

<div style="text-align: right"><span style="color:red; font-weight:bold">[Total marks for this part: 65 points]<span></div>

### <span style="color:#0b486b">4.1. One-directional RNNs for sequence modeling and neural embedding </span> ###

<div style="text-align: right"><span style="color:red; font-weight:bold">[Total marks for this part: 20 points]<span></div>

#### <span style="color:red">**Question 4.1.1**</span> 
**In this part, you need to construct an RNN to learn from the dataset of interest. Basically, you are required first to construct the class UniRNN (Uni-directional RNN) with the following requirements:**
- Attribute `data_manager (self.data_manager)`: specifies the data manager used to store data for the model.
- Attribute `cell_type (self.cell_type)`: can receive three values including `basic_rnn`, `gru`, and `lstm` which specifies the memory cells formed a hidden layer.
- `state_sizes (self.state_sizes)` indicates the list of the hidden sizes from the second hidden layers of memory cells. For example, $embed\_size =128$ and $state\_sizes = [64, 64]$ means that you have three hidden layers in your network with hidden sizes of $128, 64$ and $64$ respectively.

**Note that when declaring an embedding layer for the network, you need to set *mask_zero=True* so that the padding zeros in the sentences will be masked and ignored. This helps to have variable length RNNs. For more detail, you can refer to this [link](https://www.tensorflow.org/guide/keras/masking_and_padding).**

<div style="text-align: right"><span style="color:red">[10 points]</span></div>

In [21]:
class UniRNN:
    def __init__(self, cell_type= 'gru', embed_size= 128, state_sizes= [128, 64], data_manager= None):
        self.cell_type = cell_type
        self.state_sizes = state_sizes
        self.embed_size = embed_size
        self.data_manager = data_manager
        self.vocab_size = self.data_manager.vocab_size + 1 
        
    #return the correspoding memory cell
    @staticmethod
    def get_layer(cell_type= 'gru', state_size= 128, return_sequences= False, activation = 'tanh'):
        if cell_type=='gru':
            return tf.keras.layers.GRU(state_size, return_sequences= return_sequences, activation = activation)#Insert your code here
        elif cell_type== 'lstm':
            return tf.keras.layers.LSTM(state_size, return_sequences= return_sequences, activation = activation)#Insert your code here
        else:
            return tf.keras.layers.SimpleRNN(state_size, return_sequences= return_sequences, activation = activation)#Insert your code here
    
    def build(self):
        x = tf.keras.layers.Input(shape=[None])
        h =  tf.keras.layers.Embedding(self.vocab_size, self.embed_size)(x)#Insert your code here
        num_layers = len(self.state_sizes) #Insert your code here
        for i in range(num_layers):
            h =  self.get_layer(cell_type = self.cell_type, state_size = self.state_sizes[i], return_sequences= (i != (num_layers-1)))(h)#Insert your code here, you can insert more lines if necessary
        h = tf.keras.layers.Dense(dm.num_classes, activation='softmax')(h)
        self.model = tf.keras.Model(inputs=x, outputs=h)
   
    def compile_model(self, *args, **kwargs):
        self.model.compile(*args, **kwargs)
    
    def fit(self, *args, **kwargs):
        return self.model.fit(*args, **kwargs)
    
    def evaluate(self, *args, **kwargs):
        self.model.evaluate(*args, **kwargs)       


#### <span style="color:red">**Question 4.1.2**</span> 
**Run with basic RNN ('basic_rnn') cell with $embed\_size= 128, state\_sizes= [128, 128], data\_manager= dm$.**

<div style="text-align: right"><span style="color:red">[2 points]</span></div>

In [22]:
uni_rnn = UniRNN(cell_type= 'basic_rnn', embed_size= 128, state_sizes= [128, 128], data_manager = dm)#Insert your code here
uni_rnn.build()
opt = tf.keras.optimizers.RMSprop(learning_rate=0.0001)
uni_rnn.compile_model(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
uni_rnn.fit(dm.tf_train_set.batch(64), epochs=20, validation_data = dm.tf_valid_set.batch(64))

Epoch 1/20
7/7 [==============================] - 3s 219ms/step - loss: 1.6345 - accuracy: 0.3392 - val_loss: 1.5347 - val_accuracy: 0.3535
Epoch 2/20
7/7 [==============================] - 1s 203ms/step - loss: 1.3677 - accuracy: 0.5162 - val_loss: 1.3824 - val_accuracy: 0.4343
Epoch 3/20
7/7 [==============================] - 1s 208ms/step - loss: 1.1543 - accuracy: 0.6484 - val_loss: 1.2012 - val_accuracy: 0.5657
Epoch 4/20
7/7 [==============================] - 1s 188ms/step - loss: 0.9743 - accuracy: 0.7431 - val_loss: 1.0749 - val_accuracy: 0.6263
Epoch 5/20
7/7 [==============================] - 1s 185ms/step - loss: 0.8226 - accuracy: 0.8030 - val_loss: 0.9784 - val_accuracy: 0.6566
Epoch 6/20
7/7 [==============================] - 1s 178ms/step - loss: 0.6934 - accuracy: 0.8354 - val_loss: 0.8917 - val_accuracy: 0.6869
Epoch 7/20
7/7 [==============================] - 1s 180ms/step - loss: 0.5819 - accuracy: 0.8628 - val_loss: 0.8147 - val_accuracy: 0.7071
Epoch 8/20
7/7 [====

#### <span style="color:red">**Question 4.1.3**</span> 
**Run with GRU ('gru') cell with $embed\_size= 128, state\_sizes= [128, 128], data\_manager= dm$.**

<div style="text-align: right"><span style="color:red">[2 points]</span></div>

In [23]:
uni_rnn = UniRNN(cell_type= 'gru', embed_size= 128, state_sizes= [128, 128], data_manager = dm)#Insert your code here
uni_rnn.build()
opt = tf.keras.optimizers.RMSprop(learning_rate=0.001)
uni_rnn.compile_model(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
uni_rnn.fit(dm.tf_train_set.batch(64), epochs=20, validation_data = dm.tf_valid_set.batch(64))

Epoch 1/20
7/7 [==============================] - 4s 101ms/step - loss: 1.7087 - accuracy: 0.2394 - val_loss: 1.9591 - val_accuracy: 0.2525
Epoch 2/20
7/7 [==============================] - 0s 23ms/step - loss: 1.6711 - accuracy: 0.2843 - val_loss: 1.7815 - val_accuracy: 0.2525
Epoch 3/20
7/7 [==============================] - 0s 24ms/step - loss: 1.6438 - accuracy: 0.2668 - val_loss: 1.7690 - val_accuracy: 0.2525
Epoch 4/20
7/7 [==============================] - 0s 25ms/step - loss: 1.6410 - accuracy: 0.2668 - val_loss: 1.7626 - val_accuracy: 0.2525
Epoch 5/20
7/7 [==============================] - 0s 25ms/step - loss: 1.6397 - accuracy: 0.2668 - val_loss: 1.7582 - val_accuracy: 0.2525
Epoch 6/20
7/7 [==============================] - 0s 23ms/step - loss: 1.6389 - accuracy: 0.2668 - val_loss: 1.7547 - val_accuracy: 0.2525
Epoch 7/20
7/7 [==============================] - 0s 25ms/step - loss: 1.6382 - accuracy: 0.2668 - val_loss: 1.7518 - val_accuracy: 0.2525
Epoch 8/20
7/7 [==========

#### <span style="color:red">**Question 4.1.4**</span> 
**Run with LSTM ('lstm') cell with $embed\_size= 128, state\_sizes= [128, 128], data\_manager= dm$.**

<div style="text-align: right"><span style="color:red">[2 points]</span></div>

In [24]:
uni_rnn = UniRNN(cell_type= 'lstm', embed_size= 128, state_sizes= [128, 128], data_manager = dm)#Insert your code here
uni_rnn.build()
opt = tf.keras.optimizers.RMSprop(learning_rate=0.001)
uni_rnn.compile_model(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
uni_rnn.fit(dm.tf_train_set.batch(64), epochs=20, validation_data = dm.tf_valid_set.batch(64))

Epoch 1/20
7/7 [==============================] - 3s 118ms/step - loss: 1.7657 - accuracy: 0.2693 - val_loss: 1.7335 - val_accuracy: 0.2525
Epoch 2/20
7/7 [==============================] - 0s 24ms/step - loss: 1.6520 - accuracy: 0.2668 - val_loss: 1.8049 - val_accuracy: 0.2525
Epoch 3/20
7/7 [==============================] - 0s 25ms/step - loss: 1.6461 - accuracy: 0.2843 - val_loss: 1.7709 - val_accuracy: 0.2525
Epoch 4/20
7/7 [==============================] - 0s 26ms/step - loss: 1.6419 - accuracy: 0.2668 - val_loss: 1.7618 - val_accuracy: 0.2525
Epoch 5/20
7/7 [==============================] - 0s 26ms/step - loss: 1.6404 - accuracy: 0.2668 - val_loss: 1.7555 - val_accuracy: 0.2525
Epoch 6/20
7/7 [==============================] - 0s 24ms/step - loss: 1.6394 - accuracy: 0.2668 - val_loss: 1.7500 - val_accuracy: 0.2525
Epoch 7/20
7/7 [==============================] - 0s 27ms/step - loss: 1.6386 - accuracy: 0.2668 - val_loss: 1.7461 - val_accuracy: 0.2525
Epoch 8/20
7/7 [==========

#### <span style="color:red">**Question 4.1.5**</span> 
**Give your own comments about the performance of three memory cells for the dataset of interest as well as what happening during the training process of each cell. Note that there are not right or wrong comments and your comments rely on the status of your training. In addition, some comments and hypothesized assessments of what and why are occurring are useful to obtain the highest score for this question.**

<div style="text-align: right"><span style="color:red">[4 points]</span></div>

#Your comments here

LSTM and GRU models perform badly, they seems to fall into lcoal minimum and can not jump out. They are all under fitting. Basic rnn trains effectively, it suffers from over fitting in the end because it has larger test error.


### <span style="color:#0b486b">4.2. Bi-directional RNNs for sequence modeling and neural embedding </span> ###

<div style="text-align: right"><span style="color:red; font-weight:bold">[Total marks for this part: 20 points]<span></div>

#### <span style="color:red">**Question 4.2.1**</span> 
**In what follow, you will investigate BiRNN. The task is similar to Part 4.1 but you need to write the code for an BiRNN. Note that the function *get_layer(cell_type= 'gru', state_size= 128, return_sequences= False, activation = 'tanh')* has to return the hidden layer with bidirectional memory cells (e.g., Basic RNN, GRU, and LSTM cells).**

**Complete the code of the class *BiRNN*. Note that for the embedding layer you need to set *mask_zero=True*.**

<div style="text-align: right"><span style="color:red">[10 points]</span></div>

In [25]:
class BiRNN:
    def __init__(self, cell_type= 'gru', embed_size= 128, state_sizes= [128, 64], data_manager= None):
        self.cell_type = cell_type
        self.state_sizes = state_sizes
        self.embed_size = embed_size
        self.data_manager = data_manager
        self.vocab_size = self.data_manager.vocab_size +1
        
    @staticmethod
    def get_layer(cell_type= 'gru', state_size= 128, return_sequences= False, activation = 'tanh'):
        if cell_type=='gru':
            return tf.keras.layers.Bidirectional(
                tf.keras.layers.GRU(state_size, return_sequences= return_sequences, activation = activation)) #Insert your code here
        elif cell_type== 'lstm':
            return tf.keras.layers.Bidirectional(
                tf.keras.layers.LSTM(state_size, return_sequences= return_sequences, activation = activation))#Insert your code here
        else:
            return tf.keras.layers.Bidirectional(
                tf.keras.layers.SimpleRNN(state_size, return_sequences= return_sequences, activation = activation))#Insert your code here
    
    def build(self):
        x = tf.keras.layers.Input(shape=[None])
        h = tf.keras.layers.Embedding(self.vocab_size, self.embed_size)(x)#Insert your code here
        num_layers  = len(self.state_sizes) #Insert your code here#Insert your code here
        for i in range(num_layers):
            h = h =  self.get_layer(cell_type = self.cell_type, state_size = self.state_sizes[i], return_sequences= (i != (num_layers-1)))(h)#Insert your code here, add more lines if necessary
        h = tf.keras.layers.Dense(dm.num_classes, activation='softmax')(h)
        self.model = tf.keras.Model(inputs=x, outputs=h)
        
    
    def compile_model(self, *args, **kwargs):
        self.model.compile(*args, **kwargs)
    
    def fit(self, *args, **kwargs):
        return self.model.fit(*args, **kwargs)
    
    def evaluate(self, *args, **kwargs):
        self.model.evaluate(*args, **kwargs)       


#### <span style="color:red">**Question 4.2.2**</span> 
**Run BiRNN for basic RNN ('basic_rnn') cell with $embed\_size= 128, state\_sizes= [128, 128], data\_manager= dm$.**

<div style="text-align: right"><span style="color:red">[2 points]</span></div>

In [26]:
bi_rnn = BiRNN(cell_type= 'basic_rnn', embed_size= 128, state_sizes= [128, 128], data_manager= dm)#Insert your code here
bi_rnn.build()
opt = tf.keras.optimizers.RMSprop(learning_rate=0.001)
bi_rnn.compile_model(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
bi_rnn.fit(dm.tf_train_set.batch(64), epochs=20, validation_data = dm.tf_valid_set.batch(64))

Epoch 1/20
7/7 [==============================] - 5s 465ms/step - loss: 1.8550 - accuracy: 0.2743 - val_loss: 1.6605 - val_accuracy: 0.3939
Epoch 2/20
7/7 [==============================] - 3s 433ms/step - loss: 1.0076 - accuracy: 0.6384 - val_loss: 0.9465 - val_accuracy: 0.5657
Epoch 3/20
7/7 [==============================] - 3s 424ms/step - loss: 0.4449 - accuracy: 0.8803 - val_loss: 0.9070 - val_accuracy: 0.5960
Epoch 4/20
7/7 [==============================] - 3s 359ms/step - loss: 0.2326 - accuracy: 0.9501 - val_loss: 0.3880 - val_accuracy: 0.8687
Epoch 5/20
7/7 [==============================] - 3s 432ms/step - loss: 0.0972 - accuracy: 0.9900 - val_loss: 0.4046 - val_accuracy: 0.8687
Epoch 6/20
7/7 [==============================] - 3s 364ms/step - loss: 0.0561 - accuracy: 0.9975 - val_loss: 0.2474 - val_accuracy: 0.9495
Epoch 7/20
7/7 [==============================] - 3s 375ms/step - loss: 0.0295 - accuracy: 1.0000 - val_loss: 0.2525 - val_accuracy: 0.9091
Epoch 8/20
7/7 [====

#### <span style="color:red">**Question 4.2.3**</span> 
**Run BiRNN for GRU ('gru') cell with $embed\_size= 128, state\_sizes= [128, 128], data\_manager= dm$.**

<div style="text-align: right"><span style="color:red">[2 points]</span></div>

In [27]:
bi_rnn = BiRNN(cell_type= 'gru', embed_size= 128, state_sizes= [128, 128], data_manager= dm)#Insert your code here
bi_rnn.build()
opt = tf.keras.optimizers.RMSprop(learning_rate=0.001)
bi_rnn.compile_model(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
bi_rnn.fit(dm.tf_train_set.batch(64), epochs=20, validation_data = dm.tf_valid_set.batch(64))

Epoch 1/20
7/7 [==============================] - 5s 178ms/step - loss: 1.6174 - accuracy: 0.3242 - val_loss: 1.6574 - val_accuracy: 0.2727
Epoch 2/20
7/7 [==============================] - 0s 43ms/step - loss: 1.2220 - accuracy: 0.5686 - val_loss: 1.0755 - val_accuracy: 0.4848
Epoch 3/20
7/7 [==============================] - 0s 41ms/step - loss: 0.6971 - accuracy: 0.7556 - val_loss: 0.8696 - val_accuracy: 0.5657
Epoch 4/20
7/7 [==============================] - 0s 40ms/step - loss: 0.4593 - accuracy: 0.8279 - val_loss: 0.7856 - val_accuracy: 0.5859
Epoch 5/20
7/7 [==============================] - 0s 41ms/step - loss: 0.3051 - accuracy: 0.9177 - val_loss: 0.6464 - val_accuracy: 0.7475
Epoch 6/20
7/7 [==============================] - 0s 43ms/step - loss: 0.1824 - accuracy: 0.9501 - val_loss: 0.6095 - val_accuracy: 0.7576
Epoch 7/20
7/7 [==============================] - 0s 40ms/step - loss: 0.1013 - accuracy: 0.9825 - val_loss: 0.5273 - val_accuracy: 0.7879
Epoch 8/20
7/7 [==========

#### <span style="color:red">**Question 4.2.4**</span> 
**Run BiRNN for LSTM ('lstm') cell with $embed\_size= 128, state\_sizes= [128, 128], data\_manager= dm$.**

<div style="text-align: right"><span style="color:red">[2 points]</span></div>

In [28]:
bi_rnn = BiRNN(cell_type= 'lstm', embed_size= 128, state_sizes= [128, 128], data_manager= dm)#Insert your code here
bi_rnn.build()
opt = tf.keras.optimizers.RMSprop(learning_rate=0.001)
bi_rnn.compile_model(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
bi_rnn.fit(dm.tf_train_set.batch(64), epochs=20, validation_data = dm.tf_valid_set.batch(64))

Epoch 1/20
7/7 [==============================] - 5s 206ms/step - loss: 1.7654 - accuracy: 0.3117 - val_loss: 2.1135 - val_accuracy: 0.2525
Epoch 2/20
7/7 [==============================] - 0s 45ms/step - loss: 1.5896 - accuracy: 0.3815 - val_loss: 1.5448 - val_accuracy: 0.3434
Epoch 3/20
7/7 [==============================] - 0s 43ms/step - loss: 1.2627 - accuracy: 0.5112 - val_loss: 1.2748 - val_accuracy: 0.4040
Epoch 4/20
7/7 [==============================] - 0s 44ms/step - loss: 1.0023 - accuracy: 0.5885 - val_loss: 1.1765 - val_accuracy: 0.4141
Epoch 5/20
7/7 [==============================] - 0s 41ms/step - loss: 0.7878 - accuracy: 0.7307 - val_loss: 1.6976 - val_accuracy: 0.3939
Epoch 6/20
7/7 [==============================] - 0s 42ms/step - loss: 0.6881 - accuracy: 0.7506 - val_loss: 0.9747 - val_accuracy: 0.6263
Epoch 7/20
7/7 [==============================] - 0s 42ms/step - loss: 0.4636 - accuracy: 0.8554 - val_loss: 0.8509 - val_accuracy: 0.6970
Epoch 8/20
7/7 [==========

#### <span style="color:red">**Question 4.2.5**</span> 

**Give your own comments about the performance of three memory cells for the dataset of interest as well as comparing BiRNN to UniRNN in Part 1.**

<div style="text-align: right"><span style="color:red">[4 points]</span></div>

#Your comments here

The three models of BIRNN are all suffering from over fitting to different degree. Compared with UniRNN, BiRNN with LSTM and GRU performs much better. Compared with basic rnn of UniRNN, basic rnn of BiRNN lower test error and higher accuracy, they means BiRNN strcutre reduce over fitting problem.


### <span style="color:#0b486b">4.3. RNNs with various types, cells, and fine-tuning embedding matrix for sequence modeling and neural embedding </span> ###

<div style="text-align: right"><span style="color:red; font-weight:bold">[Total marks for this part: 20 points]<span></div>

#### <span style="color:red">**Question 4.3.1**</span> 

**In what follows, you are required to combine the code in Part 1 and Part 2 to gain a general RNN which can be either Uni-directional RNN or Bi-directional RNN and the embedding matrix can be initialized using a pretrained Word2Vect.**

**Below are the descriptions of the attributes of the class *RNN*:**
- `run_mode (self.run_mode)` has three values (scratch, init-only, and init-fine-tune).
  - `scratch` means training the embedding matrix from scratch.
  - `init-only` means only initialzing the embedding matrix with a pretrained Word2Vect but not further doing fine-tuning that matrix.
  - `init-fine-tune` means both initialzing the embedding matrix with a pretrained Word2Vect and further doing fine-tuning that matrix.
- `network_type (self.network_type)` has two values (uni-directional and bi-directional) which correspond to either Uni-directional RNN or Bi-directional RNN.
- `cell_type (self.cell_type)` has three values (simple-rnn, gru, and lstm) which specify the memory cell used in the network.
- `embed_model (self.embed_model)` specifes the pretrained Word2Vect model used.
-  `embed_size (self.embed_size)` specifes the embedding size. Note that when run_mode is either init-only' or 'init-fine-tune', this embedding size is extracted from embed_model for dimension compatability.
- `state_sizes (self.state_sizes)` indicates the list of the hidden sizes from the second hidden layers of memory cells. For example, $embed\_size =128$ and $state\_sizes = [64, 64]$ means that you have three hidden layers in your network with hidden sizes of $128, 64$ and $64$ respectively.

**Complete the code of the class *RNN*.**

<div style="text-align: right"><span style="color:red">[10 points]</span></div>

In [29]:
class RNN:
    def __init__(self, run_mode = 'scratch', cell_type= 'gru', network_type = 'uni-directional', embed_model= 'glove-wiki-gigaword-100', 
                 embed_size= 128, state_sizes = [64, 64], data_manager = None):
        self.run_mode = run_mode
        self.data_manager = data_manager
        self.cell_type = cell_type
        self.network_type = network_type
        self.state_sizes = state_sizes
        self.embed_model = embed_model
        self.embed_size = embed_size
        if self.run_mode != 'scratch':
            self.embed_size = int(self.embed_model.split("-")[-1])
        self.data_manager = data_manager
        self.vocab_size = dm.vocab_size +1
        self.word2idx = dm.word2idx
        self.word2vect = word2vect
        self.embed_matrix = np.zeros(shape= [self.vocab_size, self.embed_size])
    
    def build_embedding_matrix(self):
        if self.run_mode == 'scratch':
            return tf.keras.layers.Embedding(self.vocab_size, self.embed_size)#Insert your code here
        else:
            for i in range(1, self.vocab_size):
                word = self.data_manager.idx2word[i]
                self.embed_matrix[i] = get_word_vector(word, self.word2vect)
            return tf.keras.layers.Embedding(self.vocab_size,
                            self.embed_size,
                            weights=[self.embed_matrix],
                            trainable=(self.run_mode != 'init-only'))
    
    @staticmethod
    def get_layer(cell_type= 'gru', network_type= 'uni-directional', hidden_size= 128, return_sequences= False, activation = 'tanh'):
        if network_type == 'uni-directional':
            if cell_type=='gru':
                return tf.keras.layers.Bidirectional(
                    tf.keras.layers.GRU(hidden_size, return_sequences= return_sequences, activation = activation)) #Insert your code here
            elif cell_type== 'lstm':
                return tf.keras.layers.Bidirectional(
                    tf.keras.layers.LSTM(hidden_size, return_sequences= return_sequences, activation = activation))#Insert your code here
            else:
                return tf.keras.layers.Bidirectional(
                    tf.keras.layers.SimpleRNN(hidden_size, return_sequences= return_sequences, activation = activation))#Insert your code here
        else:
            if cell_type=='gru':
                return tf.keras.layers.Bidirectional(
                    tf.keras.layers.GRU(hidden_size, return_sequences= return_sequences, activation = activation)) #Insert your code here
            elif cell_type== 'lstm':
                return tf.keras.layers.Bidirectional(
                    tf.keras.layers.LSTM(hidden_size, return_sequences= return_sequences, activation = activation))#Insert your code here
            else:
                return tf.keras.layers.Bidirectional(
                    tf.keras.layers.SimpleRNN(hidden_size, return_sequences= return_sequences, activation = activation))#Insert your code here
    
    def build(self):
        x = tf.keras.layers.Input(shape=[None])
        h = self.build_embedding_matrix()(x)#Insert your code here
        num_layers = len(self.state_sizes) #Insert your code here#Insert your code here
        for i in range(num_layers):
            h = h =  self.get_layer(cell_type = self.cell_type, network_type = self.network_type, hidden_size = self.state_sizes[i], return_sequences= (i != (num_layers-1)))(h)#Insert your code here, add more lines if necessary
        h = tf.keras.layers.Dense(dm.num_classes, activation='softmax')(h)
        self.model = tf.keras.Model(inputs=x, outputs=h)#Insert your code here
        
    def compile_model(self, *args, **kwargs):
        self.model.compile(*args, **kwargs)
    
    def fit(self, *args, **kwargs):
        return self.model.fit(*args, **kwargs)
    
    def evaluate(self, *args, **kwargs):
        self.model.evaluate(*args, **kwargs)       


In [30]:
bi_rnn = RNN(run_mode = 'scratch', cell_type= 'gru', network_type = 'uni-directional', embed_model= 'glove-wiki-gigaword-100', 
                 embed_size= 128, state_sizes = [64, 64], data_manager = dm)#Insert your code here
bi_rnn.build()
opt = tf.keras.optimizers.RMSprop(learning_rate=0.001)
bi_rnn.compile_model(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
bi_rnn.fit(dm.tf_train_set.batch(64), epochs=20, validation_data = dm.tf_valid_set.batch(64))

Epoch 1/20
7/7 [==============================] - 5s 195ms/step - loss: 1.6629 - accuracy: 0.3092 - val_loss: 1.6227 - val_accuracy: 0.2727
Epoch 2/20
7/7 [==============================] - 0s 33ms/step - loss: 1.3567 - accuracy: 0.4663 - val_loss: 1.2784 - val_accuracy: 0.4040
Epoch 3/20
7/7 [==============================] - 0s 33ms/step - loss: 0.9764 - accuracy: 0.6334 - val_loss: 0.9913 - val_accuracy: 0.5051
Epoch 4/20
7/7 [==============================] - 0s 31ms/step - loss: 0.6772 - accuracy: 0.7855 - val_loss: 0.7916 - val_accuracy: 0.6263
Epoch 5/20
7/7 [==============================] - 0s 33ms/step - loss: 0.4575 - accuracy: 0.8753 - val_loss: 0.7273 - val_accuracy: 0.6566
Epoch 6/20
7/7 [==============================] - 0s 34ms/step - loss: 0.3300 - accuracy: 0.9102 - val_loss: 0.6699 - val_accuracy: 0.7273
Epoch 7/20
7/7 [==============================] - 0s 30ms/step - loss: 0.2529 - accuracy: 0.9401 - val_loss: 0.5568 - val_accuracy: 0.7879
Epoch 8/20
7/7 [==========

#### <span style="color:red">**Question 4.3.2**</span> 

**Design the experiment to compare three running modes. Note that you should stick with fixed values for other attributes and only vary *run_mode*. Give your comments for the results.**

<div style="text-align: right"><span style="color:red">[5 points]</span></div>

comments: use embed_model such as glove-wiki-gigaword-100 can relieve the problem of over fitting. In my experiment, 'init-only' mode seems to perform best. This is because the pre embed model is good enough and do not need to optimize, so 'init-fine-tune' performs worse than 'init-only' mode.

In [31]:
bi_rnn = RNN(run_mode = 'init-only', cell_type= 'gru', network_type = 'uni-directional', embed_model= 'glove-wiki-gigaword-100', 
                 embed_size= 128, state_sizes = [64, 64], data_manager = dm)#Insert your code here
bi_rnn.build()
opt = tf.keras.optimizers.RMSprop(learning_rate=0.001)
bi_rnn.compile_model(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
bi_rnn.fit(dm.tf_train_set.batch(64), epochs=20, validation_data = dm.tf_valid_set.batch(64))

Epoch 1/20
7/7 [==============================] - 5s 190ms/step - loss: 1.3017 - accuracy: 0.6060 - val_loss: 1.0905 - val_accuracy: 0.4949
Epoch 2/20
7/7 [==============================] - 0s 32ms/step - loss: 0.8067 - accuracy: 0.7631 - val_loss: 0.8302 - val_accuracy: 0.5152
Epoch 3/20
7/7 [==============================] - 0s 34ms/step - loss: 0.5900 - accuracy: 0.8304 - val_loss: 0.6700 - val_accuracy: 0.6667
Epoch 4/20
7/7 [==============================] - 0s 33ms/step - loss: 0.4513 - accuracy: 0.8853 - val_loss: 0.5620 - val_accuracy: 0.7374
Epoch 5/20
7/7 [==============================] - 0s 32ms/step - loss: 0.3566 - accuracy: 0.8928 - val_loss: 0.4853 - val_accuracy: 0.7475
Epoch 6/20
7/7 [==============================] - 0s 31ms/step - loss: 0.2891 - accuracy: 0.9202 - val_loss: 0.4244 - val_accuracy: 0.7879
Epoch 7/20
7/7 [==============================] - 0s 33ms/step - loss: 0.2383 - accuracy: 0.9302 - val_loss: 0.3727 - val_accuracy: 0.8384
Epoch 8/20
7/7 [==========

In [32]:
bi_rnn = RNN(run_mode = 'init-fine-tune', cell_type= 'gru', network_type = 'uni-directional', embed_model= 'glove-wiki-gigaword-100', 
                 embed_size= 128, state_sizes = [64, 64], data_manager = dm)#Insert your code here
bi_rnn.build()
opt = tf.keras.optimizers.RMSprop(learning_rate=0.001)
bi_rnn.compile_model(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
bi_rnn.fit(dm.tf_train_set.batch(64), epochs=20, validation_data = dm.tf_valid_set.batch(64))

Epoch 1/20
7/7 [==============================] - 6s 182ms/step - loss: 1.3494 - accuracy: 0.6160 - val_loss: 1.0827 - val_accuracy: 0.4949
Epoch 2/20
7/7 [==============================] - 0s 33ms/step - loss: 0.7986 - accuracy: 0.7606 - val_loss: 0.8224 - val_accuracy: 0.5556
Epoch 3/20
7/7 [==============================] - 0s 33ms/step - loss: 0.5645 - accuracy: 0.8429 - val_loss: 0.6503 - val_accuracy: 0.6465
Epoch 4/20
7/7 [==============================] - 0s 35ms/step - loss: 0.4134 - accuracy: 0.9027 - val_loss: 0.5297 - val_accuracy: 0.7879
Epoch 5/20
7/7 [==============================] - 0s 33ms/step - loss: 0.3084 - accuracy: 0.9202 - val_loss: 0.4445 - val_accuracy: 0.8081
Epoch 6/20
7/7 [==============================] - 0s 33ms/step - loss: 0.2350 - accuracy: 0.9377 - val_loss: 0.3828 - val_accuracy: 0.8081
Epoch 7/20
7/7 [==============================] - 0s 34ms/step - loss: 0.1807 - accuracy: 0.9551 - val_loss: 0.3369 - val_accuracy: 0.8485
Epoch 8/20
7/7 [==========

#### <span style="color:red">**Question 4.3.3**</span> 

**Run the above general RNN with at least five parameter sets and try to obtain the best performance. You can stick with the running mode *init-fine-tune* and use grid search to tune other parameters. Record your best model which will be used in the next part.**

<div style="text-align: right"><span style="color:red">[5 points]</span></div>

In [63]:
bi_rnn = RNN(run_mode = 'init-fine-tune', cell_type= 'lstm', network_type = 'bi-directional', embed_model= 'glove-wiki-gigaword-100', 
                 embed_size= 128, state_sizes = [64,64], data_manager = dm)#Insert your code here
bi_rnn.build()
opt = tf.keras.optimizers.Adam(learning_rate=8e-3)
# opt = tf.keras.optimizers.RMSprop(learning_rate=0.001)
bi_rnn.compile_model(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
bi_rnn.fit(dm.tf_train_set.batch(32), epochs=20, validation_data = dm.tf_valid_set.batch(32))

Epoch 1/20
13/13 [==============================] - 5s 120ms/step - loss: 1.0311 - accuracy: 0.6010 - val_loss: 0.4434 - val_accuracy: 0.8889
Epoch 2/20
13/13 [==============================] - 0s 31ms/step - loss: 0.3900 - accuracy: 0.8828 - val_loss: 0.4967 - val_accuracy: 0.7879
Epoch 3/20
13/13 [==============================] - 0s 31ms/step - loss: 0.1881 - accuracy: 0.9377 - val_loss: 0.0834 - val_accuracy: 0.9798
Epoch 4/20
13/13 [==============================] - 0s 33ms/step - loss: 0.0945 - accuracy: 0.9676 - val_loss: 0.0564 - val_accuracy: 0.9899
Epoch 5/20
13/13 [==============================] - 0s 32ms/step - loss: 0.0268 - accuracy: 0.9975 - val_loss: 0.0185 - val_accuracy: 1.0000
Epoch 6/20
13/13 [==============================] - 0s 31ms/step - loss: 0.0068 - accuracy: 1.0000 - val_loss: 0.0068 - val_accuracy: 1.0000
Epoch 7/20
13/13 [==============================] - 0s 32ms/step - loss: 0.0021 - accuracy: 1.0000 - val_loss: 0.0044 - val_accuracy: 1.0000
Epoch 8/20
1

#Report your results here

best model: RNN(run_mode = 'init-fine-tune', cell_type= 'lstm', network_type = 'bi-directional', embed_model= 'glove-wiki-gigaword-100', 
                 embed_size= 128, state_sizes = [64,64], data_manager = dm)

accuracy = 1.0000

In [71]:
#The run of your best model here
my_best_rnn = RNN(run_mode = 'init-fine-tune', cell_type= 'lstm', network_type = 'bi-directional', embed_model= 'glove-wiki-gigaword-100', 
                 embed_size= 128, state_sizes = [64,64], data_manager = dm)#Insert your code here
my_best_rnn.build()
opt = tf.keras.optimizers.Adam(learning_rate=8e-3)
# opt = tf.keras.optimizers.RMSprop(learning_rate=0.001)
my_best_rnn.compile_model(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
my_best_rnn.fit(dm.tf_train_set.batch(32), epochs=20, validation_data = dm.tf_valid_set.batch(32))

Epoch 1/20
13/13 [==============================] - 5s 116ms/step - loss: 1.0188 - accuracy: 0.6060 - val_loss: 0.5651 - val_accuracy: 0.8687
Epoch 2/20
13/13 [==============================] - 0s 31ms/step - loss: 0.3327 - accuracy: 0.8928 - val_loss: 0.3341 - val_accuracy: 0.8788
Epoch 3/20
13/13 [==============================] - 0s 31ms/step - loss: 0.1344 - accuracy: 0.9576 - val_loss: 0.1847 - val_accuracy: 0.9293
Epoch 4/20
13/13 [==============================] - 0s 30ms/step - loss: 0.1239 - accuracy: 0.9551 - val_loss: 0.1746 - val_accuracy: 0.9293
Epoch 5/20
13/13 [==============================] - 0s 31ms/step - loss: 0.1267 - accuracy: 0.9626 - val_loss: 0.0963 - val_accuracy: 0.9798
Epoch 6/20
13/13 [==============================] - 0s 31ms/step - loss: 0.0548 - accuracy: 0.9800 - val_loss: 0.0682 - val_accuracy: 0.9798
Epoch 7/20
13/13 [==============================] - 0s 31ms/step - loss: 0.0201 - accuracy: 0.9950 - val_loss: 0.0930 - val_accuracy: 0.9697
Epoch 8/20
1

### <span style="color:#0b486b">4.4. Investigating the embedding vectors from the embedding matrix</span> ###

<div style="text-align: right"><span style="color:red; font-weight:bold">[Total marks for this part: 5 points]<span></div>

**As you know, the embedding matrix is a collection of embedding vectors, each is for one word. In this part, you will base on the cosine similarity of the embedding vectors for the words to find the top-k most relevant words for a given word.**

**Good embeddings should have words close in meaning near each other by some similarity metrics. The similarity metric we'll use is the `consine` distance, which is defined for two vector $\mathbf{u}$ and $\mathbf{v}$ as $\cos(\mathbf{u}, \mathbf{v})=\frac{\mathbf{u} \cdot \mathbf{v}}{\left\Vert{\mathbf{u}}\right\Vert\left\Vert{\mathbf{v}}\right\Vert}$ where $\cdot$ means dot product and $\left\Vert\cdot\right\Vert$ means the $\mathcal{L}_2$ norm.**

In [72]:
def cosine_similarity(u,v):
    return np.dot(u,v)/(np.linalg.norm(u)*np.linalg.norm(u))

#### <span style="color:red">**Question 4.4.1**</span> 

**You are required to write the code for the function *find_most_similar(word= None, k=5, model= None)*. As its name, this function returns the top-k most relevant word for a given word based on the cosine similarity of the embedding vectors.**

<div style="text-align: right"><span style="color:red">[5 points]</span></div>

In [75]:
from dis import dis
from turtle import distance


def find_most_similar(word= None, k=5, model= None):
    try:
        result = []
        distance = [0 for i in range(1, bi_rnn.vocab_size)]
        idx = model.data_manager.word2idx[word]
        for i in range(1, model.vocab_size):
            distance[i-1] = cosine_similarity(bi_rnn.embed_matrix[idx], bi_rnn.embed_matrix[i])
        distance2 = distance.copy()
        distance.sort(reverse=True)
        for i in range(k+1):
            idx2 = distance2.index(distance[i])
            if idx2 != idx:
                result.append(model.data_manager.idx2word[idx2])
        print(result)

        #Insert your code here
    except: #word not in the vocabulary
        print("Word is not in the dictionary!")

Here is the example of the above function. As you can observe, the result makes sense which demonstrates that we obtain a good model with the meaningful embedding matrix.

In [76]:
find_most_similar(word='poland', k=10, model= my_best_rnn)

['sonar', '1957', 'be', 'archives', 'term', 'spoken', 'engines', 'pink', 'baby', 'leader', 'smallest']


--- 
<div style="text-align: center"> <span style="color:black">END OF ASSIGNMENT</span> </div>